Goal: To scrap Data from Reddit handle r/dataengineering after scrolling till certain no_of_days

And then find order the data in upvotes

then load the data in Postgres

## First step: To scrape data for last 7 days I need to scroll down the webpage

I need to install Selenium to scroll data in webpage

For webdrivers: we need Selenium and a webdriver

pip install selenium

pip install webdriver-manager

First import and install chrome webdriver-manager

In [86]:
# For Chrome Driver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") # linux only
# chrome_options.add_argument("--headless")

driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# For finding and waiting for elements
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait


/tmp/ipykernel_21325/57819732.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [87]:
import time

## Fixing the URL
## Change the subbreddit handle you need

In [88]:
url_domain = "https://www.reddit.com"
# subreddit_url = "/r/dataengineering/"
subreddit_url = '/r/Python/'

Now pass the URL you need to open in the driver

In [89]:
driver.get(url_domain + subreddit_url)

The below code will scroll the websit until certains old post arrives

In [90]:
wait = WebDriverWait(driver, 3)

pending_list = {3, 4}
found = False
count = 0
scroll_from = 0
scroll_to = 3000
while not found:
    driver.execute_script(f"window.scrollTo({scroll_from},{scroll_from + scroll_to})")
    scroll_from += scroll_to
#     print(pending_list, scroll_from)
    found = not pending_list
    try:
        done_set = set()
        for k in pending_list:
            find_elem = wait.until(EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{k} days ago')]")))
            if find_elem: done_set.add(k)
        pending_list = pending_list - done_set
    except TimeoutException:
        pass
    time.sleep(3)

In [91]:
with open('scrapped_data.txt', 'w') as f:
    f.write(driver.page_source)

In [79]:
driver.close()

## Second Step: Now we need to scrap the data from the page

In [2]:
from bs4 import BeautifulSoup

In [3]:
with open('scrapped_data.txt', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [4]:
parent_elem = soup.find("div", attrs={"tabindex":"0"}).parent
elem = parent_elem.findChildren("div", recursive=False)[3].findChildren("div", recursive=False)

# data = soup.find("div", class_="rpBJOHq2PR60pnwJlUyP0")
data = parent_elem.findChildren("div", recursive=False)[3].select("div[style]", recursive=False)

In [5]:
data = []
for e in elem:
    link = upvote = None
    link_elem = e.find("a", {"data-click-id": "body"})
    if link_elem:
        link = link_elem.get('href')
    button = e.find("button")
    if button:
        upvote = button.parent.find("div").text
    span = e.find('span', {"data-click-id": "timestamp"})
    title = e.find('h3')
    if link and upvote and span and title:
#         pass
        data_dict = {'link': link,
                    'upvote': upvote,
                    'time': span.text,
                    'title': title.text}
        data.append(data_dict)

## Step3: Use Pandas to Tranform Data

In [63]:
import pandas as pd

In [64]:
df = pd.DataFrame(data)

In [65]:
df['upvote'] = df['upvote'].replace('Vote', 0)
# to convert upvote to Integer
df['upvote'] = df['upvote'].astype(int)

In [66]:
# To categorize time as D-Days, H-Hours, M-Minutes, S-Seconds
def categorize_time(x):
    if 'day' in x:
        x = x.split()
        return f'{x[0]} D'
    elif 'hour' in x:
        x = x.split()
        return f'{x[0]} H'
    elif 'minute' in x:
        x = x.split()
        return f'{x[0]} M'
    elif 'second' in x:
        x = x.split()
        return f'{x[0]} S'
    else:
        return None
df['time_category'] = df['time'].apply(categorize_time)

In [67]:
# After Categorizing time, we are reducing the time if difference in days, if diffence in time like H, M, S then default as today
def time_diff(x):
    t, c = x.split()
    t = int(t)
    if c in ('H', 'M', 'S'):
        return pd.Timestamp.now().floor('d')
    elif c in ('D'):
        return pd.Timestamp.now().floor('d') - pd.Timedelta(days = t)
    else:
        return None
df['post_date'] = df['time_category'].apply(time_diff)

In [68]:
# Adding the domain to construct workable link
df['link'] = df['link'].apply(lambda x: url_domain + x)

In [69]:
df_to_be_loaded = df[['title', 'upvote', 'link', 'post_date']].sort_values(by='upvote', ascending=False)

## Step3: Load the Data

In [70]:
df_to_be_loaded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 61 to 20
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      77 non-null     object        
 1   upvote     77 non-null     int64         
 2   link       77 non-null     object        
 3   post_date  77 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.0+ KB


In [71]:
df_to_be_loaded.to_csv('reddit_upvote.csv')

In [72]:
import psycopg2
from psycopg2 import Error

In [85]:
try:
    # Connect to an existing database
    conn = psycopg2.connect(user="python_psql",
                                  password="password",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="reddit_scrapper")

    cursor = conn.cursor()

    # Create table if not exists
    query = '''create table if not exists scrapped_data(
            id serial primary key,
            title varchar(300),
            upvote int,
            link varchar(300),
            post_date date
        )'''
    cursor.execute(query)
    conn.commit()

    # Convert date-time to string to upload in sql
    # df_to_be_loaded['post_date'] = df_to_be_loaded['post_date'].dt.strftime('%d-%m-%Y')
    data_to_be_inserted = df_to_be_loaded.to_numpy().tolist()
    query = "INSERT INTO scrapped_data(title, upvote, link, post_date) VALUES(%s, %s, %s, %s)"
    cursor.executemany(query,data_to_be_inserted)
    conn.commit()

    cursor.close()
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)